## SRJ

## Using ROHF

In [ ]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
rhf.kernel()

converged SCF energy = -1655.34968383623


np.float64(-1655.3496838362307)

In [ ]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc') #Kinetic energy integrals + Nuclear energy integrals = Hcore
two_ao = mol_pyscf.intor('int2e_sph') # Two electron integrals

print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)  
print(two_ao.shape)

one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)


two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])

#H_fermionic_full = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

#H_full = qml.jordan_wigner(H_fermionic_full)

(27, 27)
(27, 27)
(27, 27, 27, 27)


In [ ]:
import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)


core=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] 
active=[18, 19, 20, 21, 22, 23]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)

The value of core: [120.55734926]
Total orbitals are 24
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p

# Old form of converting full space to active space

active_indices = [18, 19, 20, 21, 22, 23]

one_mo_active = one_mo[np.ix_(active_indices, active_indices)]

import numpy as np

two_mo_active = two_mo[np.ix_(active_indices, active_indices, active_indices, active_indices)]

core_constant_active = core_constant  # Usually unchanged for active space

import pennylane as qml

H_fermionic_active = qml.qchem.fermionic_observable(core_constant_active, one_mo_active, two_mo_active)
H_active = qml.jordan_wigner(H_fermionic_active)


## Using hf_state similar to qiskit

In [4]:




#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    #hf_state = qchem.hf_state(active_electrons, qubits)
    hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=50) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1655.1484203124342
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.10812497583517312 for operator a⁺(0) a⁺(1) a(10) a(11)
Highest gradient excitation is [0, 1, 10, 11]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.25584954]
Final cost: -1655.1553689998254
 0: ──X─╭FermionicDoubleExcitation(0.26)─┤  State
 1: ──X─├FermionicDoubleExcitation(0.26)─┤  State
 2: ──X─│────────────────────────────────┤  State
 6: ──X─│────────────────────────────────┤  State
 7: ──X─│────────────────────────────────┤  State
 8: ──X─│────────────────────────────────┤  State
10: ────├FermionicDoubleExcitation(0.26)─┤  State
11: ────╰FermionicDoubleExcitation(0.26)─┤  State
The adapt iteration now is 1


...

The highest operator value is 0.00011993710652047785 for operator a⁺(0) a(10)
Highest gradient excitation is [0, 10]
Final updated parameters: [ 2.62164272e-01 -1.57535039e+00  7.31515626e-01 -1.09746734e-01
  3.71069272e-02  8.02995876e-03 -2.40747439e+00 -1.99178315e-02
  8.14185033e-03  1.68724450e-02  7.90926048e-03  7.21900467e-03
 -1.46981386e-03  3.46053709e-03 -9.21842641e-03 -5.82166864e-03
 -1.24680264e-03  8.05955775e-04 -7.64855678e-04 -3.09079050e-04
 -5.56666099e-04  1.94438572e-05 -3.96864383e-06  2.05410345e-05
  7.05808781e-06  2.27264278e-05 -1.85159968e-05  1.04016337e-05
  2.35862544e-05 -1.76694439e-05  1.83937673e-05  3.32820171e-05
  3.08244840e-05  2.56272926e-05  4.16543543e-06  2.42564952e-05
  7.46607165e-06  2.17559598e-05  2.40741554e-05  8.26331666e-06
  1.61492889e-05  3.11918801e-06  1.41206260e-05  1.62270500e-05
  1.10952747e-17  8.19984185e-18  5.80827067e-18  2.42172761e-20
  1.15543038e-20]
Final cost: -1655.2741574864322
 0: ──X─╭FermionicDoubleExc

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.00011993710652039632 for operator a⁺(0) a(10)
Highest gradient excitation is [0, 10]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [ 2.62164272e-01 -1.57535039e+00  7.31515626e-01 -1.09746734e-01
  3.71069272e-02  8.02995876e-03 -2.40747439e+00 -1.99178315e-02
  8.14185033e-03  1.68724450e-02  7.90926048e-03  7.21900467e-03
 -1.46981386e-03  3.46053709e-03 -9.21842641e-03 -5.82166864e-03
 -1.24680264e-03  8.05955775e-04 -7.64855678e-04 -3.09079050e-04
 -5.56666099e-04  1.94438572e-05 -3.96864383e-06  2.05410345e-05
  7.05808781e-06  2.27264278e-05 -1.85159968e-05  1.04016337e-05
  2.35862544e-05 -1.76694439e-05  1.83937673e-05  3.32820171e-05
  3.08244840e-05  2.56272926e-05  4.16543543e-06  2.42564952e-05
  7.46607165e-06  2.17559598e-05  2.40741554e-05  8.26331666e-06
  1.61492889e-05  3.11918801e-06  1.41206260e-05  1.62270500e-05
  1.10952747e-17  8.20406989e-18  5.82095479e-18  3.69014005e-20
  1.57823452e-20  8.45608298e-21]
Final cost: -1655.2741574864317
 0: ──X─╭FermionicDoubleExcitation(0.26)────────────────────────────────── ···
 1: ──X─├FermionicDoubleExcitation(0.26)─────────

In [5]:
energies

[-1655.1553689998254,
 -1655.2654931663938,
 -1655.267099866241,
 -1655.2673328431435,
 -1655.2673791225584,
 -1655.26739356342,
 -1655.2741233804677,
 -1655.2741391515005,
 -1655.2741447819071,
 -1655.2741513190222,
 -1655.2741527144076,
 -1655.2741538837295,
 -1655.2741538837195,
 -1655.274154545813,
 -1655.2741558783357,
 -1655.2741565888446,
 -1655.2741570548071,
 -1655.2741572498192,
 -1655.2741573971243,
 -1655.274157412174,
 -1655.27415747875,
 -1655.27415747874,
 -1655.274157478731,
 -1655.2741574787206,
 -1655.2741574787112,
 -1655.2741574787015,
 -1655.2741574786905,
 -1655.274157478682,
 -1655.274157478671,
 -1655.274157478661,
 -1655.2741574786523,
 -1655.2741574786414,
 -1655.2741574786317,
 -1655.2741574786219,
 -1655.2741574786119,
 -1655.2741574786014,
 -1655.2741574785928,
 -1655.2741574785825,
 -1655.2741574785732,
 -1655.274157478563,
 -1655.2741574785518,
 -1655.2741574785414,
 -1655.2741574785314,
 -1655.2741574864356,
 -1655.2741574864344,
 -1655.2741574864335,
 -

In [6]:
ash_excitation

[[0, 1, 10, 11],
 [4, 5, 6, 7],
 [2, 5, 6, 9],
 [0, 10],
 [4, 10],
 [0, 1, 6, 7],
 [3, 4, 7, 8],
 [4, 10],
 [0, 1, 8, 9],
 [1, 11],
 [1, 2, 6, 9],
 [1, 4, 8, 9],
 [0, 1, 10, 11],
 [0, 1, 10, 11],
 [1, 2, 7, 8],
 [1, 4, 6, 7],
 [2, 3, 10, 11],
 [0, 10],
 [4, 5, 10, 11],
 [1, 4, 6, 7],
 [1, 4, 6, 7],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [1, 2, 7, 8],
 [0, 10],
 [0, 10],
 [0, 10],
 [0, 10],
 [0, 10],
 [0, 10]]